Python Imports

In [ ]:
%pip install --upgrade pip
%pip install pandas
%pip install numpy
%pip install plotly
%pip install nbformat

In [2]:
import pandas as pd
import numpy as np
from itertools import groupby
from operator import itemgetter
import plotly.graph_objs as go
import plotly.express as px

Setup and Data Structuring

In [ ]:
# pandas settings
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 0)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.expand_frame_repr', False)

# Defining variables and functions
sensors = ["dig1", "ana1", "noise"]
def visualize_dataframe(df, sample_size=None):
    # Downsample the DataFrame if sample_size is provided
    if sample_size:
        downsampled_df = df.sample(n=sample_size).sort_values(by=['Time'])
    else:
        downsampled_df = df
    
    # Melt the DataFrame
    melted_df = downsampled_df.melt(id_vars=['Time'], value_vars=['ana1', 'dig1', 'noise'], var_name='Sensor', value_name='Value')
    
    # Create a Plotly figure
    fig = px.line(melted_df, x='Time', y='Value', color='Sensor', title='Sensor Data Over Time')
    
    # Update layout
    fig.update_layout(
        xaxis_title='Time',
        yaxis_title='Value',
        hovermode='x unified'
    )
    
    # Show the figure
    fig.show()

# Load the data from the CSV file
df = pd.read_csv('_initial_dataset.csv', comment='#', low_memory=False)

# Drop the unnecessary columns
df = df.drop(columns=['Unnamed: 0', 'result', '_start', '_stop', '_field', 'table', 'host', 'topic'])
df = df.rename(columns={'_time': 'Time', '_value': 'Value', '_measurement': 'Sensor', 'serial': 'Index', 'unit': 'Unit'})

# Reorder the columns
columns_order = ['Index', 'Sensor', 'Value', 'Unit', 'Time']
df = df[columns_order]

# Convert the data types for easier reading
# df['Index'] = df['Index'].astype(str)
# df['Index'] = df['Index'].str.zfill(6)
# df['Time'] = pd.to_datetime(df['Time']).dt.strftime('%d.%m.%Y %H:%M:%S')

# Sort the data by the index and the sensor
sensor_order = {'dig1': 1, 'ana1': 2, 'noise': 3}
df['SensorOrder'] = df['Sensor'].map(sensor_order)
df = df.sort_values(by=['Index', 'SensorOrder']).drop(columns=['SensorOrder'])

# Set the Index column as the actual table-index
df.set_index('Index', inplace=True)

print(df.head())

Search the DataFrame for mismatches in Sensordata/Availability for the same indexes

In [ ]:
#TODO: move this logic to datacleaning and apply it to the compressed dataframe
expected_sensors = {'dig1', 'ana1', 'noise'}
# Identify rows with missing sensor values
missing_sensors = []
for idx, group in df.groupby(df.index):
    present_sensors = set(group['Sensor'])
    missing = expected_sensors - present_sensors
    for sensor in missing:
        missing_sensors.append({'Index': idx, 'Missing Sensor': sensor})

# Create a DataFrame to display missing sensors
missing_sensors_df = pd.DataFrame(missing_sensors)

# Print the indices with missing sensor values
print("Indices with missing sensor values:")
print(missing_sensors_df)

Pivot the DataFrame to Save 2/3 of the Space

In [ ]:
# Pivot the DataFrame
time_df = df[df['Sensor'] == 'dig1'][['Time']].reset_index()
pivot_df = df.pivot_table(index=df.index, columns='Sensor', values='Value').reset_index()
pivot_df = pivot_df.merge(time_df, on='Index', how='left')
pivot_df.set_index('Index', inplace=True)

# Display the pivoted DataFrame
print(pivot_df.head())

pivot_df.to_csv('1datasetCompressed.csv')

small_df = pivot_df

Get a List with Pairs of Datapoints with from/to values of Missing IDs

In [ ]:
all_indexes = pd.Series(range(small_df.index.min(), small_df.index.max() + 1))
missing_indexes = all_indexes[~all_indexes.isin(small_df.index)]

# Find ranges of missing indexes
missing_ranges = []
for k, g in groupby(enumerate(missing_indexes), lambda x: x[0] - x[1]):
    group = list(map(itemgetter(1), g))
    missing_ranges.append((group[0], group[-1]))

# Create a DataFrame with the required columns
missing_data = []
for start, end in missing_ranges:
    first_missing_date = small_df.loc[start - 1, 'Time'] if start - 1 in small_df.index else None
    last_missing_date = small_df.loc[end + 1, 'Time'] if end + 1 in small_df.index else None
    missing_data.append([start, end, first_missing_date, last_missing_date])

missing_df = pd.DataFrame(missing_data, columns=['First Missing Index', 'Last Missing Index', 'First Missing Time', 'Last Missing Time'])

print(missing_df.head())

# Visualize the DataFrame
first_time = small_df['Time'].iloc[0]
last_time = small_df['Time'].iloc[-1]
